!pip uninstall -y scopyon
!pip install git+https://github.com/ecell/scopyon
!pip freeze | grep scopyon

In [1]:
seed = 123
num_samples = 30
exposure_time = 33.0e-3
interval = 33.0e-3
num_frames = 3
Nm = [100, 100, 100]
Dm = [0.222e-12, 0.032e-12, 0.008e-12]
transmat = [
    [0.0, 0.5, 0.0],
    [0.5, 0.0, 0.2],
    [0.0, 1.0, 0.0]]

In [2]:
nproc = 20

In [3]:
# !pip install mlflow

In [4]:
import numpy
rng = numpy.random.RandomState(seed)

In [5]:
import scopyon

In [6]:
config = scopyon.DefaultConfiguration()
config.default.effects.photo_bleaching.switch = False
config.default.detector.exposure_time = exposure_time
pixel_length = config.default.detector.pixel_length / config.default.magnification
L_2 = config.default.detector.image_size[0] * pixel_length * 0.5
L_2

1.6995850622406638e-05

In [7]:
config.environ.processes = nproc

In [8]:
timepoints = numpy.linspace(0, interval * num_frames, num_frames + 1)
ndim = 2

In [9]:
import pathlib
artifacts = pathlib.Path("./artifacts")
artifacts.mkdir(parents=True, exist_ok=True)

In [10]:
config.save(artifacts / 'config.yaml')

In [18]:
for i in range(num_samples):
    samples = scopyon.sample(timepoints, N=Nm, lower=-L_2, upper=+L_2, ndim=ndim, D=Dm, transmat=transmat, rng=rng)
    inputs = [(t, numpy.hstack((points[:, : ndim], points[:, [ndim + 1]], numpy.ones((points.shape[0], 1), dtype=numpy.float64)))) for t, points in zip(timepoints, samples)]
    ret = list(scopyon.generate_images(inputs, num_frames=num_frames, config=config, rng=rng, full_output=True))
    
    inputs_ = []
    for t, data in inputs:
        inputs_.extend(([t] + list(row) for row in data))
    inputs_ = numpy.array(inputs_)
    numpy.save(artifacts / f"inputs{i:03d}.npy", inputs_)

    numpy.save(artifacts / f"images{i:03d}.npy", numpy.array([img.as_array() for img, infodict in ret]))

    true_data = []
    for t, (_, infodict) in zip(timepoints, ret):
        true_data.extend([t, key] + list(value) for key, value in infodict['true_data'].items())
    true_data = numpy.array(true_data)
    numpy.save(artifacts / f"true_data{i:03d}.npy", true_data)

In [22]:
!ls ./artifacts

analysis1.ipynb    images004.npy  inputs004.npy     true_data001.npy
analysis2.ipynb    images005.npy  inputs005.npy     true_data002.npy
config.yaml	   images006.npy  inputs006.npy     true_data003.npy
evaluation1.ipynb  images007.npy  inputs007.npy     true_data004.npy
generation.ipynb   images008.npy  inputs008.npy     true_data005.npy
images000.npy	   inputs000.npy  spots000.npy	    true_data006.npy
images001.npy	   inputs001.npy  spots001.npy	    true_data007.npy
images002.npy	   inputs002.npy  spots002.npy	    true_data008.npy
images003.npy	   inputs003.npy  true_data000.npy
